https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')
raw_html = raw_html.read()
raw_html

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Natural language processing - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YCzpnugtsyR7zGxW90850QAAAMU","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Natural_language_processing","wgTitle":"Natural language processing","wgCurRevisionId":1007284878,"wgRevisionId":1007284878,"wgArticleId":21652,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: location","Articles with short description","Short description matches Wikidata","Commons link from Wikidata","Wikiped

In [2]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')
article_text = ''


In [3]:
for para in article_paragraphs:
    article_text += para.text
article_text

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The result is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves. \nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated i

In [4]:
corpus = nltk.sent_tokenize(article_text)
corpus

['Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
 'The result is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.',
 'The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.',
 'Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.',
 'Natural language processing has its roots in the 1950s.',
 'Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involve

In [5]:
#We then pre-process it to remove all the special characters and multiple empty spaces.
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])
    
corpus

['natural language processing nlp is a subfield of linguistics computer science and artificial intelligence concerned with the interactions between computers and human language in particular how to program computers to process and analyze large amounts of natural language data ',
 'the result is a computer capable of understanding the contents of documents including the contextual nuances of the language within them ',
 'the technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves ',
 'challenges in natural language processing frequently involve speech recognition natural language understanding and natural language generation ',
 'natural language processing has its roots in the 1950s ',
 'already in 1950 alan turing published an article titled computing machinery and intelligence which proposed what is now called the turing test as a criterion of intelligence a task that involves the automate

In [6]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
wordfreq
#but it wont remove special chaarcteers like , ) ' etc....

{'natural': 20,
 'language': 28,
 'processing': 16,
 'nlp': 16,
 'is': 15,
 'a': 25,
 'subfield': 1,
 'of': 68,
 'linguistics': 9,
 'computer': 4,
 'science': 3,
 'and': 27,
 'artificial': 2,
 'intelligence': 4,
 'concerned': 1,
 'with': 11,
 'the': 69,
 'interactions': 1,
 'between': 1,
 'computers': 2,
 'human': 2,
 'in': 27,
 'particular': 1,
 'how': 2,
 'to': 28,
 'program': 1,
 'process': 2,
 'analyze': 1,
 'large': 3,
 'amounts': 1,
 'data': 5,
 'result': 1,
 'capable': 1,
 'understanding': 4,
 'contents': 1,
 'documents': 4,
 'including': 1,
 'contextual': 1,
 'nuances': 1,
 'within': 1,
 'them': 1,
 'technology': 1,
 'can': 5,
 'then': 2,
 'accurately': 1,
 'extract': 1,
 'information': 1,
 'insights': 1,
 'contained': 1,
 'as': 16,
 'well': 2,
 'categorize': 1,
 'organize': 1,
 'themselves': 1,
 'challenges': 1,
 'frequently': 2,
 'involve': 2,
 'speech': 4,
 'recognition': 2,
 'generation': 2,
 'has': 6,
 'its': 2,
 'roots': 1,
 '1950s': 1,
 'already': 1,
 '1950': 1,
 'alan':

In [7]:
#   OR WE CAN DO IT WITH GENSIM KIND OF WAY....AS SHOWN IN BELOW TWO CELLS

In [8]:
from nltk.tokenize import regexp_tokenize
tokenized_sen_of_sen = [regexp_tokenize(s, "\w+") for s in corpus]
print("\n\ntokenized lines:\n",tokenized_sen_of_sen)



tokenized lines:
 [['natural', 'language', 'processing', 'nlp', 'is', 'a', 'subfield', 'of', 'linguistics', 'computer', 'science', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', 'language', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data'], ['the', 'result', 'is', 'a', 'computer', 'capable', 'of', 'understanding', 'the', 'contents', 'of', 'documents', 'including', 'the', 'contextual', 'nuances', 'of', 'the', 'language', 'within', 'them'], ['the', 'technology', 'can', 'then', 'accurately', 'extract', 'information', 'and', 'insights', 'contained', 'in', 'the', 'documents', 'as', 'well', 'as', 'categorize', 'and', 'organize', 'the', 'documents', 'themselves'], ['challenges', 'in', 'natural', 'language', 'processing', 'frequently', 'involve', 'speech', 'recognition', 'natural', 'language', 'understanding', 'and', 'nat

In [9]:
# Import Dictionary
from gensim.corpora.dictionary import Dictionary

# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(tokenized_sen_of_sen)
print(dictionary)
dictionary.token2id   #displays the ids of each word and NOT THE FREQUENCY

Dictionary(526 unique tokens: ['a', 'amounts', 'analyze', 'and', 'artificial']...)


{'a': 0,
 'amounts': 1,
 'analyze': 2,
 'and': 3,
 'artificial': 4,
 'between': 5,
 'computer': 6,
 'computers': 7,
 'concerned': 8,
 'data': 9,
 'how': 10,
 'human': 11,
 'in': 12,
 'intelligence': 13,
 'interactions': 14,
 'is': 15,
 'language': 16,
 'large': 17,
 'linguistics': 18,
 'natural': 19,
 'nlp': 20,
 'of': 21,
 'particular': 22,
 'process': 23,
 'processing': 24,
 'program': 25,
 'science': 26,
 'subfield': 27,
 'the': 28,
 'to': 29,
 'with': 30,
 'capable': 31,
 'contents': 32,
 'contextual': 33,
 'documents': 34,
 'including': 35,
 'nuances': 36,
 'result': 37,
 'them': 38,
 'understanding': 39,
 'within': 40,
 'accurately': 41,
 'as': 42,
 'can': 43,
 'categorize': 44,
 'contained': 45,
 'extract': 46,
 'information': 47,
 'insights': 48,
 'organize': 49,
 'technology': 50,
 'themselves': 51,
 'then': 52,
 'well': 53,
 'challenges': 54,
 'frequently': 55,
 'generation': 56,
 'involve': 57,
 'recognition': 58,
 'speech': 59,
 '1950s': 60,
 'has': 61,
 'its': 62,
 'roots'

In [10]:
corpu = [dictionary.doc2bow(cor) for cor in tokenized_sen_of_sen]    #converts the dictionary into a corpus or bag of words
corpu                             #corpu is a list of lists..input and output is both a list of lits

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 3),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 3),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 2),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 1)],
 [(0, 1),
  (6, 1),
  (15, 1),
  (16, 1),
  (21, 3),
  (28, 4),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1)],
 [(3, 2),
  (12, 1),
  (28, 3),
  (34, 2),
  (41, 1),
  (42, 2),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(3, 1),
  (12, 1),
  (16, 3),
  (19, 3),
  (24, 1),
  (39, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(12, 1),
  (16, 1),
  (19, 1),
  (24, 1),
  (28, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1)],
 [(0, 3),
  (3, 2),
  (4, 1),
  (12, 1),
  (13, 3),
  (15, 1),
  (16, 1),


In [12]:
from gensim.models import TfidfModel
doc = corpu[4]

# Create a new TfidfModel using the corpus: tfidf
tfidf = TfidfModel(corpu)

# Calculate the tfidf weights of doc: tfidf_weights
tfidf_weights = tfidf[doc]

# Print the first five weights
print(tfidf_weights[:5])

# Sort the weights from highest to lowest: sorted_tfidf_weights
sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)

# Print the top 5 weighted words
for term_id, weight in sorted_tfidf_weights:
    print(dictionary.get(term_id), weight)

[(12, 0.13084940663921613), (16, 0.11639016352214945), (19, 0.15567693239081873), (24, 0.1650009323904069), (28, 0.051091033338939267)]
1950s 0.5562379739107542
roots 0.5562379739107542
its 0.45609771353077033
has 0.29737915601604337
processing 0.1650009323904069
natural 0.15567693239081873
in 0.13084940663921613
language 0.11639016352214945
the 0.051091033338939267


BEFORE CONTINUATION......................OF 

In [44]:
###filter the top 200 most frequently occurring words.
import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)
most_freq

[',',
 'of',
 'the',
 '.',
 'and',
 'to',
 'language',
 'in',
 'a',
 '[',
 ']',
 '(',
 ')',
 'NLP',
 'is',
 'natural',
 'processing',
 'as',
 'that',
 'statistical',
 'with',
 'on',
 'linguistics',
 'rules',
 'tasks',
 'machine',
 'for',
 'are',
 'models',
 'cognitive',
 'learning',
 'neural',
 'such',
 'have',
 '``',
 'The',
 'has',
 'by',
 'e.g.',
 'systems',
 'algorithms',
 'methods',
 'many',
 'research',
 'input',
 'data',
 "''",
 'can',
 'an',
 'which',
 ':',
 'been',
 'they',
 'more',
 'computer',
 'understanding',
 'documents',
 'from',
 'symbolic',
 'or',
 'based',
 'grammar',
 'machine-learning',
 'results',
 'be',
 'Cognitive',
 'Natural',
 'science',
 'intelligence',
 'large',
 'task',
 '1980s',
 'however',
 'was',
 'computational',
 'In',
 'deep',
 'set',
 'More',
 'commonly',
 'used',
 'Since',
 'through',
 'real-world',
 'when',
 'larger',
 'use',
 'given',
 'approaches',
 'translation',
 'trends',
 'aspects',
 'artificial',
 'computers',
 'human',
 'how',
 'process',
 '

The next step is to find the IDF values for the most frequently occurring words in the corpus. The following script does that:

In [45]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))
word_idf_values

{',': 0.23922968906583406,
 'of': 0.2947995402206448,
 'the': 0.3536400402435784,
 '.': 0.0,
 'and': 0.8544153281560676,
 'to': 0.6720937713621129,
 'language': 0.8544153281560676,
 'in': 0.9057086225436182,
 'a': 1.0169342576538425,
 '[': 1.2110902720948,
 ']': 1.2110902720948,
 '(': 1.285198244248522,
 ')': 1.2110902720948,
 'NLP': 1.2110902720948,
 'is': 1.1420974006078486,
 'natural': 1.2110902720948,
 'processing': 1.2110902720948,
 'as': 1.285198244248522,
 'that': 1.3652409519220583,
 'statistical': 1.285198244248522,
 'with': 1.6529230243738393,
 'on': 1.7707060600302227,
 'linguistics': 1.9042374526547452,
 'rules': 1.9042374526547452,
 'tasks': 1.6529230243738393,
 'machine': 1.9042374526547452,
 'for': 1.547562508716013,
 'are': 1.547562508716013,
 'models': 1.7707060600302227,
 'cognitive': 1.9042374526547452,
 'learning': 1.9042374526547452,
 'neural': 1.7707060600302227,
 'such': 1.6529230243738393,
 'have': 1.6529230243738393,
 '``': 1.7707060600302227,
 'The': 1.9042374

In the script above, we create an empty dictionary word_idf_values. This dictionary will store most frequently occurring words as keys and their corresponding IDF values as dictionary values. Next, we iterate through the list of most frequently occurring words. During each iteration, we create a variable doc_containing_word. This variable will store the number of documents in which the word appears. Next, we iterate through all the sentences in our corpus. The sentence is tokenized and then we check if the word exists in the sentence or not, if the word exists, we increment the doc_containing_word variable. Finally, to calculate the IDF value we divide the total number of sentences by the total number of documents containing the word.

The next step is to create the TF dictionary for each word. In the TF dictionary, the key will be the most frequently occuring words, while values will be 49 dimensional vectors since our document has 49 sentences. Each value in the vector will belong to the TF value of the word for the corresponding sentence. Look at the following script:

In [46]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector
word_tf_values

{',': [0.06666666666666667,
  0.04,
  0.0,
  0.1111111111111111,
  0.0,
  0.05172413793103448,
  0.05172413793103448,
  0.05263157894736842,
  0.07407407407407407,
  0.0,
  0.045454545454545456,
  0.06944444444444445,
  0.08333333333333333,
  0.03076923076923077,
  0.044444444444444446,
  0.0,
  0.0,
  0.14814814814814814,
  0.025,
  0.08,
  0.10869565217391304,
  0.0,
  0.04878048780487805,
  0.05555555555555555,
  0.05555555555555555,
  0.0,
  0.04,
  0.058823529411764705,
  0.05405405405405406,
  0.04,
  0.0,
  0.0,
  0.038461538461538464,
  0.05555555555555555,
  0.0,
  0.05555555555555555,
  0.023809523809523808,
  0.03125,
  0.08695652173913043,
  0.05263157894736842,
  0.045454545454545456,
  0.043478260869565216,
  0.057971014492753624,
  0.11688311688311688,
  0.10714285714285714,
  0.043478260869565216,
  0.0],
 'of': [0.044444444444444446,
  0.12,
  0.0,
  0.0,
  0.0,
  0.034482758620689655,
  0.034482758620689655,
  0.05263157894736842,
  0.037037037037037035,
  0.09375,
  

In the above script, we create a dictionary that contains the word as the key and a list of 49 items as a value since we have 49 sentences in our corpus. Each item in the list stores the TF value of the word for the corresponding sentence. In the script above word_tf_values is our dictionary. For each word, we create a list sent_tf_vector.

We then iterate through each sentence in the corpus and tokenize the sentence. The word from the outer loop is matched with each word in the sentence. If a match is found the doc_freq variable is incremented by 1. Once, all the words in the sentence are iterated, the doc_freq is divided by the total length of the sentence to find the TF value of the word for that sentence. This process repeats for all the words in the most frequently occurring word list. The final word_tf_values dictionary will contain 200 words as keys. For each word, there will be a list of 49 items as the value

You can see that the word is the key whereas a list of 49 items is the value for each key.

Now we have IDF values of all the words, along with TF values of every word across the sentences. The next step is to simply multiply IDF values with TF values.

In [47]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)
    
tfidf_values

[[0.01594864593772227,
  0.009569187562633362,
  0.0,
  0.02658107656287045,
  0.0,
  0.012373949434439693,
  0.012373949434439693,
  0.012591036266622843,
  0.0177207177085803,
  0.0,
  0.01087407677571973,
  0.016613172851794034,
  0.019935807422152837,
  0.007360913509717971,
  0.010632430625148181,
  0.0,
  0.0,
  0.0354414354171606,
  0.005980742226645852,
  0.019138375125266723,
  0.026003227072373267,
  0.0,
  0.011669740930040686,
  0.013290538281435225,
  0.013290538281435225,
  0.0,
  0.009569187562633362,
  0.014072334650931415,
  0.012931334544099138,
  0.009569187562633362,
  0.0,
  0.0,
  0.009201141887147464,
  0.013290538281435225,
  0.0,
  0.013290538281435225,
  0.005695944977757953,
  0.007475927783307314,
  0.020802581657898615,
  0.012591036266622843,
  0.01087407677571973,
  0.010401290828949307,
  0.01386838777193241,
  0.027961911708993592,
  0.02563175239991079,
  0.010401290828949307,
  0.0],
 [0.013102201787584214,
  0.03537594482647737,
  0.0,
  0.0,
  0.0,


In the above script, we create a list called tfidf_values. We then iterated through all the keys in the word_tf_values dictionary. These keys are basically the most frequently occurring words. Using these words, we retrieve the 49-dimensional list that contains the TF values for the word corresponding to each sentence. Next, the TF value is multiplied by the IDF value of the word and stored in the tf_idf_score variable. The variable is then appended to the tf_idf_sentences list. Finally, the tf_idf_sentences list is appended to the tfidf_values list.

Now at this point in time, the tfidf_values is a list of lists. Where each item is a 49-dimensional list that contains TFIDF values of a particular word for all the sentences.